In [12]:
%%capture
%run 05_block.ipynb
%run 06_mine.ipynb

Account state holds the current balance and nonce of an account.

In [511]:
class AccountState:
    def __init__(self): self.balance,self.nonce = 0,0
    def inc_nonce  (self):        self.nonce   += 1
    def add_balance(self, value): self.balance += value
    def sub_balance(self, value): self.balance -= value
    def __str__(self): return f'balance: {self.balance}\nnonce: {self.nonce}'

The state stores the dictionary `state` which stores the `AccountState` for each address. A genesis block is used to initialize state the first time.

The state is updated with txs. `apply` validates if the tx is correct and then updates each `AccountState` accordingly.

In [512]:
class State:
    def __init__(self): self.state={}
    def new   (self, pub): self.state[pub] = AccountState()
    def is_new(self, pub): return pub not in self.state
    def add(self, pubs):
        for pub in pubs:
            if self.is_new(pub): self.new(pub)
    
    def init(self, genesis):
        assert genesis.number    == 0
        assert genesis.prev_hash == '0x0'
        miner = genesis.miner
        for tx in genesis.txs:
            self.add([tx.fr, tx.to, miner])
            assert val_sig(tx)
            self.state[tx.fr].inc_nonce()
            self.state[tx.to].add_balance(tx.value)
        self.state[miner].add_balance(genesis.reward)
        
    def apply(self, tx, miner):
        self.add([tx.fr,tx.to,miner])
        assert val_sig(tx)
        assert self.state[tx.fr].nonce  == tx.nonce
        assert self.state[tx.fr].balance - tx.value > 0
        self.state[tx.fr].inc_nonce()
        self.state[tx.fr].sub_balance(tx.value)
        self.state[tx.to].add_balance(tx.value)
        self.state[miner].add_balance(tx.fee)
        return True
    
    def apply_reward(self, miner, reward):
        self.state[miner].add_balance(reward)
            
    def __str__(self):
        return '\n'.join(f'{ph(k)}\n{v}\n' for k,v in self.state.items())

## Blockchain

Consists of a number of blocks. Each block points to the block that came before it. Imagine a linked list of blocks linked by its `prev_hash`.

A `candidate` block is one that could be used for mining.

In [513]:
class Blockchain:
    def __init__(self, genesis):
        self.state  = State()
        self.blocks = []
        self.state.init(genesis)
        self.blocks.append(genesis)

    def candidate(self, txs):
        return Block(txs, self.blocks[-1].hash, len(self.blocks))
        
    def add(self, mb):
        assert self.val(mb)
        for tx in mb.txs: assert self.state.apply(tx, mb.miner)
        self.state.apply_reward(mb.miner, mb.reward)
        self.blocks.append(mb)
        return True
    
    def val(self, mb): 
        assert val_pow(mb)
        assert mb.number == len(self.blocks)
        if len(self.blocks) > 0: assert self.blocks[-1].hash == mb.prev_hash
        return True
    
    def __str__(self): return ('\n'+'--'*20+'\n').join(f'{str(block)}' for block in self.blocks)

In [514]:
acc1,acc2,acc3 = Account(),Account(),Account()
miner = Miner(acc3)

In [515]:
tx1 = acc1.signed_tx(acc2, 24, 0.1)
tx2 = acc2.signed_tx(acc1, 18, 0.12)

gb = miner.mine_genesis([tx1, tx2], reward=200)

In [516]:
bc = Blockchain(gb); print(bc)

time:		Sat Mar 27 23:39:27 2021
number:		0
prev_hash:	0x0

txs:		

time:	Sat Mar 27 23:39:27 2021
from:	👵 0x0f60e737abc61A...18C
to:	👵 0x0f60e737abc61A...18C
value:	200 ether
fee:	0 ether
nonce:	0
hash:	📼 960267a748b29038...df6
signed:	true

time:	Sat Mar 27 23:39:27 2021
from:	🔷 0xD17f62c13e7EA1...114
to:	🔑 0xABf0e9FEcE6a74...2F2
value:	24 ether
fee:	0.1 ether
nonce:	0
hash:	💋 254bbd4ac34daf4d...b95
signed:	true

time:	Sat Mar 27 23:39:27 2021
from:	🔑 0xABf0e9FEcE6a74...2F2
to:	🔷 0xD17f62c13e7EA1...114
value:	18 ether
fee:	0.12 ether
nonce:	0
hash:	📻 95ae0bd01bd9d08b...5f5
signed:	true

miner:		👵 0x0f60e737abc61A...18C
hash:		📭 876fd8e6184cdf27...b6f
diff:		0
reward:		200
nonce:		0


In [517]:
tx1 = acc1.signed_tx(acc2, 3, 0.05)
tx2 = acc2.signed_tx(acc1, 9, 0.16)

block = bc.candidate([tx1,tx2])
mb    = miner.mine(block, 1, 100)
bc.add(mb); print(bc)

time:		Sat Mar 27 23:39:27 2021
number:		0
prev_hash:	0x0

txs:		

time:	Sat Mar 27 23:39:27 2021
from:	👵 0x0f60e737abc61A...18C
to:	👵 0x0f60e737abc61A...18C
value:	200 ether
fee:	0 ether
nonce:	0
hash:	📼 960267a748b29038...df6
signed:	true

time:	Sat Mar 27 23:39:27 2021
from:	🔷 0xD17f62c13e7EA1...114
to:	🔑 0xABf0e9FEcE6a74...2F2
value:	24 ether
fee:	0.1 ether
nonce:	0
hash:	💋 254bbd4ac34daf4d...b95
signed:	true

time:	Sat Mar 27 23:39:27 2021
from:	🔑 0xABf0e9FEcE6a74...2F2
to:	🔷 0xD17f62c13e7EA1...114
value:	18 ether
fee:	0.12 ether
nonce:	0
hash:	📻 95ae0bd01bd9d08b...5f5
signed:	true

miner:		👵 0x0f60e737abc61A...18C
hash:		📭 876fd8e6184cdf27...b6f
diff:		0
reward:		200
nonce:		0
----------------------------------------
time:		Sat Mar 27 23:39:29 2021
number:		1
prev_hash:	📭 876fd8e6184cdf27...b6f

txs:		

time:	Sat Mar 27 23:39:29 2021
from:	👵 0x0f60e737abc61A...18C
to:	👵 0x0f60e737abc61A...18C
value:	100 ether
fee:	0 ether
nonce:	1
hash:	🕟 f9961e528499b651...c90
signed:	true

time

In [518]:
acc4 = Account()
tx1 = acc2.signed_tx(acc4, 12, 1.5)

block = bc.candidate([tx1])
mb    = miner.mine(block, 2, 100)
bc.add(mb); print(bc)

time:		Sat Mar 27 23:39:27 2021
number:		0
prev_hash:	0x0

txs:		

time:	Sat Mar 27 23:39:27 2021
from:	👵 0x0f60e737abc61A...18C
to:	👵 0x0f60e737abc61A...18C
value:	200 ether
fee:	0 ether
nonce:	0
hash:	📼 960267a748b29038...df6
signed:	true

time:	Sat Mar 27 23:39:27 2021
from:	🔷 0xD17f62c13e7EA1...114
to:	🔑 0xABf0e9FEcE6a74...2F2
value:	24 ether
fee:	0.1 ether
nonce:	0
hash:	💋 254bbd4ac34daf4d...b95
signed:	true

time:	Sat Mar 27 23:39:27 2021
from:	🔑 0xABf0e9FEcE6a74...2F2
to:	🔷 0xD17f62c13e7EA1...114
value:	18 ether
fee:	0.12 ether
nonce:	0
hash:	📻 95ae0bd01bd9d08b...5f5
signed:	true

miner:		👵 0x0f60e737abc61A...18C
hash:		📭 876fd8e6184cdf27...b6f
diff:		0
reward:		200
nonce:		0
----------------------------------------
time:		Sat Mar 27 23:39:29 2021
number:		1
prev_hash:	📭 876fd8e6184cdf27...b6f

txs:		

time:	Sat Mar 27 23:39:29 2021
from:	👵 0x0f60e737abc61A...18C
to:	👵 0x0f60e737abc61A...18C
value:	100 ether
fee:	0 ether
nonce:	1
hash:	🕟 f9961e528499b651...c90
signed:	true

time